In [1]:
import pandas as pd
import datetime
import numpy as np
import os
import glob
from os.path import basename
import calendar

In [2]:
os.getcwd()

'/home/jian/Projects/Saatva/Transaction_Data'

# 2018

In [3]:
data_1=pd.read_csv("/home/jian/Projects/Saatva/Transaction_Data/3 Brands_transactions_20180101-20180308.csv",
                   dtype=str,usecols=['Order ID','Date','Time','Shipping Zip','Product Revenue','Order Type','Brand'])
data_1=data_1[data_1['Brand']=="Saatva"]
print(data_1['Order Type'].unique())
data_1['Date']=data_1['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())

['phone' 'online']


In [4]:
data_2=pd.read_csv("/home/jian/Projects/Saatva/Transaction_Data/Saatva_Only_transactions_60 days-20180403.csv",
                   dtype=str,usecols=['Order ID','Date','Time','Shipping Zip','Product Revenue','Order Type','Brand'])
data_2=data_2[data_2['Brand']=="Saatva"]
print(data_2['Order Type'].unique())
data_2['Date']=data_2['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())

['online' 'phone']


In [5]:
weekly_file_list=sorted(glob.glob("/home/jian/Projects/Saatva/Transaction_Data/New Klipfolio with phone num and rep name/*.csv"),reverse=True)
aggregated_sales_data=pd.DataFrame(columns=['Order ID','Date','Time','Shipping Zip','Product Revenue','Order Type','Brand'])
for file in weekly_file_list:
    weekly_download=pd.read_csv(file,dtype=str,usecols=['Order ID','Date','Time','Shipping Zip','Product Revenue','Order Type','Brand'])
    weekly_download=weekly_download[weekly_download['Brand']=="Saatva"]
    try:
        weekly_download['Date']=weekly_download['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())
    except:
        weekly_download['Date']=weekly_download['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%B-%Y").date())
    
    weekly_download=weekly_download[weekly_download['Date']!=weekly_download['Date'].max()]
    weekly_download=weekly_download[weekly_download['Date']!=weekly_download['Date'].min()]
    weekly_download=weekly_download[~weekly_download['Date'].isin(aggregated_sales_data['Date'])]
    aggregated_sales_data=aggregated_sales_data.append(weekly_download)

In [6]:
data_2=data_2[data_2['Date']!=data_2['Date'].max()]
data_2=data_2[data_2['Date']!=data_2['Date'].min()]
data_2=data_2[~data_2['Date'].isin(aggregated_sales_data['Date'])]
aggregated_sales_data=aggregated_sales_data.append(data_2)

In [7]:
data_1=data_1[data_1['Date']!=data_2['Date'].max()]
data_1=data_1[~data_1['Date'].isin(aggregated_sales_data['Date'])]
aggregated_sales_data=aggregated_sales_data.append(data_1)

In [8]:
aggregated_sales_data['Product Revenue']=aggregated_sales_data['Product Revenue'].astype(float)
print(aggregated_sales_data['Order ID'].apply(lambda x: x[0]).unique())
print(aggregated_sales_data['Order ID'].apply(lambda x: len(x)).unique())

['6' '5']
[6]


In [9]:
Saatva_Sales_by_day=aggregated_sales_data.groupby(['Date',"Brand"])['Product Revenue'].sum().to_frame().reset_index()
Saatva_Trans_by_day=aggregated_sales_data.groupby(['Date',"Brand"])['Product Revenue'].count().to_frame().reset_index()

Saatva_Sales_by_day=Saatva_Sales_by_day.rename(columns={"Product Revenue":"Sales"})
Saatva_Trans_by_day=Saatva_Trans_by_day.rename(columns={"Product Revenue":"Trans"})

Saatva_Sales_by_day=pd.merge(Saatva_Sales_by_day,Saatva_Trans_by_day,on=['Date','Brand'],how="left")
Saatva_Sales_by_day['weekday']=Saatva_Sales_by_day['Date'].apply(lambda x: calendar.day_name[x.weekday()])

In [10]:
writer=pd.ExcelWriter("/home/jian/Projects/Saatva/Transaction_Data/sales_by_day/Saatva_Aggregated_Sales_to_Daily_till_20181205_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
Saatva_Sales_by_day.to_excel(writer,'Saatva_sales_by_day_aggregated',index=False)
aggregated_sales_data.to_excel(writer,'all_data_aggregated',index=False)
writer.save()

# 2017

In [11]:
Saatva_2017_sales_data=pd.read_excel("/home/jian/Projects/Saatva/Transaction_Data/Order_data.xlsx",sheetname="Saatva",dtype=str)
Saatva_2017_sales_data=Saatva_2017_sales_data[['order_id','date','time','zipcode','product_revenue','online/phone?','brand']]
Saatva_2017_sales_data['date']=Saatva_2017_sales_data['date'].apply(lambda x: datetime.datetime.strptime(x.split(" ")[0],"%Y-%m-%d").date())
Saatva_2017_sales_data['product_revenue']=Saatva_2017_sales_data['product_revenue'].astype(float)

In [12]:
Saatva_Sales_by_day=Saatva_2017_sales_data.groupby(['date',"brand"])['product_revenue'].sum().to_frame().reset_index()
Saatva_Trans_by_day=Saatva_2017_sales_data.groupby(['date',"brand"])['product_revenue'].count().to_frame().reset_index()

Saatva_Sales_by_day_2017=Saatva_Sales_by_day.rename(columns={"product_revenue":"Sales"})
Saatva_Trans_by_day_2017=Saatva_Trans_by_day.rename(columns={"product_revenue":"Trans"})

Saatva_Sales_by_day_2017=pd.merge(Saatva_Sales_by_day_2017,Saatva_Trans_by_day_2017,on=['date','brand'],how="left")
Saatva_Sales_by_day_2017['weekday']=Saatva_Sales_by_day_2017['date'].apply(lambda x: calendar.day_name[x.weekday()])

In [13]:
writer=pd.ExcelWriter("/home/jian/Projects/Saatva/Transaction_Data/sales_by_day/Saatva_Aggregated_Sales_to_Daily_2017_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
Saatva_Sales_by_day_2017.to_excel(writer,'Saatva_sales_by_day_2017',index=False)
Saatva_2017_sales_data.to_excel(writer,'sales_data_2017',index=False)
writer.save()